In [2]:
import pandas as pd
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
import numpy as np


In [3]:
dataset = pd.read_pickle("./prepared_ad.pkl")


In [4]:
train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)
train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features.pop('PRICE')
test_labels = test_features.pop('PRICE')

In [5]:
normalizer = tf.keras.layers.experimental.preprocessing.Normalization(axis=-1)
normalizer.adapt(np.array(train_features))


In [6]:
def build_and_compile_model(norm):
  model = keras.Sequential([
      norm,
      layers.Dense(64, activation='relu'),
      layers.Dense(64, activation='relu'),
      layers.Dense(1)
  ])

  model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.00005))
  return model

In [7]:
dnn_model = build_and_compile_model(normalizer)

In [8]:
history = dnn_model.fit(
    train_features,
    train_labels,
    validation_split=0.2,
    epochs=1500)

Epoch 1/1500
2/2 [==============================] - 0s 106ms/step - loss: 52.8605 - val_loss: 56.2293
Epoch 2/1500
2/2 [==============================] - 0s 15ms/step - loss: 52.8488 - val_loss: 56.2186
Epoch 3/1500
2/2 [==============================] - 0s 16ms/step - loss: 52.8369 - val_loss: 56.2080
Epoch 4/1500
2/2 [==============================] - 0s 16ms/step - loss: 52.8258 - val_loss: 56.1974
Epoch 5/1500
2/2 [==============================] - 0s 15ms/step - loss: 52.8140 - val_loss: 56.1869
Epoch 6/1500
2/2 [==============================] - 0s 15ms/step - loss: 52.8028 - val_loss: 56.1764
Epoch 7/1500
2/2 [==============================] - 0s 21ms/step - loss: 52.7912 - val_loss: 56.1658
Epoch 8/1500
2/2 [==============================] - 0s 16ms/step - loss: 52.7799 - val_loss: 56.1553
Epoch 9/1500
2/2 [==============================] - 0s 19ms/step - loss: 52.7684 - val_loss: 56.1449
Epoch 10/1500
2/2 [==============================] - 0s 14ms/step - loss: 52.7569 - val_lo

In [9]:
dnn_model.save('price_predictor_model')


INFO:tensorflow:Assets written to: price_predictor_model/assets


In [10]:
model = keras.models.load_model('price_predictor_model')


In [11]:
predicted = model.predict(test_features)